In [45]:
import branca.colormap as cm  # Add this import

# Rest of your imports
import geopandas as gpd
from shapely.geometry import shape
import openrouteservice
from openrouteservice import exceptions
import pandas as pd
import folium
from folium.plugins import HeatMap

# Initialize ORS client and define generate_isochrones function
# (As shown in the previous section)

In [ ]:
flats_with_pop=gpd.read_file('/home/silas/projects/msc_thesis/data/derived_data/flats_population.gpkg')
flats_with_pop.to_crs(epsg=4326, inplace=True)
rcps=gpd.read_file('/home/silas/projects/msc_thesis/data/raw_data/geodata_stadt_Zuerich/recycling_sammelstellen/data/stzh.poi_sammelstelle_view.shp')
rcps.to_crs(epsg=4326, inplace=True)

In [ ]:
def generate_isochrones(client, rcps, time_range):
    """
    Generate isochrones for each RCP point.

    Parameters:
    - client: ORS client instance.
    - rcps: GeoDataFrame with RCP points.
    - time_range: List of time ranges in seconds.

    Returns:
    - GeoDataFrame with isochrones and time attribute.
    """
    isochrones_list = []

    for idx, row in rcps.iterrows():
        lon, lat = row.geometry.x, row.geometry.y
        for time in time_range:
            try:
                params = {
                    "locations": [[lon, lat]],
                    "range": [time],
                    "range_type": "time",
                    "units": "m",
                    "location_type": "start",
                    "smoothing": 0.3,
                    "profile": "foot-walking",
                }
                isochrone = client.isochrones(**params)
                for feature in isochrone['features']:
                    isochrones_list.append({
                        'geometry': shape(feature['geometry']),
                        'rcp_id': row['standort_i'],
                        'time': time / 60})
                    print(f"Generated isochrone for RCP ID {row['standort_i']} for {time} minutes.")

            except exceptions.ApiError as e:
                print(f"API error for RCP ID {row['standort_i']} at time {time} seconds: {e}")
            except Exception as e:
                print(f"Unexpected error for RCP ID {row['standort_i']} at time {time} seconds: {e}")

    isochrones_gdf = gpd.GeoDataFrame(isochrones_list, crs="EPSG:4326")
    return isochrones_gdf

def get_ors_client():
    """Initialize OpenRouteService client"""
    api_key = os.getenv("ORS_API_KEY")
    if not api_key:
        logger.error("OpenRouteService API key not found.")
        sys.exit(1)
    return openrouteservice.Client(key=api_key)

client = get_ors_client()

# Example usage
time_range = list(range(60, 660, 60))  # [60, 120, ..., 600] seconds
isochrones_gdf = generate_isochrones(client, rcps, time_range)
print(isochrones_gdf)

/home/silas/miniconda3/envs/geo_env/lib/python3.13/site-packages/openrouteservice/client.py:211: UserWarning: Rate limit exceeded. Retrying for the 1st time.
  warnings.warn('Rate limit exceeded. Retrying for the {0}{1} time.'.format(retry_counter + 1,
/home/silas/miniconda3/envs/geo_env/lib/python3.13/site-packages/openrouteservice/client.py:211: UserWarning: Rate limit exceeded. Retrying for the 2nd time.
  warnings.warn('Rate limit exceeded. Retrying for the {0}{1} time.'.format(retry_counter + 1,
/home/silas/miniconda3/envs/geo_env/lib/python3.13/site-packages/openrouteservice/client.py:211: UserWarning: Rate limit exceeded. Retrying for the 3rd time.
  warnings.warn('Rate limit exceeded. Retrying for the {0}{1} time.'.format(retry_counter + 1,
/home/silas/miniconda3/envs/geo_env/lib/python3.13/site-packages/openrouteservice/client.py:211: UserWarning: Rate limit exceeded. Retrying for the 4th time.
  warnings.warn('Rate limit exceeded. Retrying for the {0}{1} time.'.format(retry_c

In [ ]:
isochrones_gdf
print(isochrones_gdf.crs)

In [ ]:
import folium
from folium.plugins import MarkerCluster

# Create a folium map centered around Zurich
m = folium.Map(location=[47.3769, 8.5417], zoom_start=13)

# Add isochrones to the map
for _, row in isochrones_gdf.iterrows():
    folium.GeoJson(row['geometry'], name=f"Isochrone {row['time']} min").add_to(m)

# Add recycling collection points to the map
marker_cluster = MarkerCluster().add_to(m)
for _, row in rcps.iterrows():
    folium.Marker(
        location=[row.geometry.y, row.geometry.x],
        popup=row['adresse'],
        icon=folium.Icon(color='green', icon='recycle', prefix='fa')
    ).add_to(marker_cluster)

# Display the map
m

In [57]:
from shapely.ops import unary_union

def merge_isochrones_preserve_time(isochrones_gdf):
    """
    Merge isochrones preserving lower time values.

    Parameters:
    - isochrones_gdf: GeoDataFrame with isochrones and 'time' attribute.

    Returns:
    - GeoDataFrame with merged isochrones.
    """
    # Ensure CRS is EPSG:4326
    if isochrones_gdf.crs != "EPSG:4326":
        isochrones_gdf = isochrones_gdf.to_crs(epsg=4326)

    # Sort isochrones by 'time' ascending
    isochrones_sorted = isochrones_gdf.sort_values(by='time')

    merged_isochrones = gpd.GeoDataFrame(columns=isochrones_sorted.columns, crs="EPSG:4326")

    # Initialize an empty geometry for subtraction
    accumulated_geom = None

    for _, row in isochrones_sorted.iterrows():
        current_geom = row.geometry
        current_time = row['time']

        if accumulated_geom:
            remaining_geom = current_geom.difference(accumulated_geom)
        else:
            remaining_geom = current_geom

        if not remaining_geom.is_empty:
            new_row = row.copy()
            new_row.geometry = remaining_geom
            # Ensure the new_row GeoDataFrame has the correct CRS
            new_row = gpd.GeoDataFrame([new_row], crs="EPSG:4326")
            merged_isochrones = pd.concat([merged_isochrones, new_row], ignore_index=True)
            # Update accumulated geometry
            if accumulated_geom:
                accumulated_geom = unary_union([accumulated_geom, remaining_geom])
            else:
                accumulated_geom = remaining_geom
        print(f"Processed isochrone with time {current_time} seconds.")

    return merged_isochrones

# Merge isochrones
merged_isochrones_gdf = merge_isochrones_preserve_time(isochrones_gdf)
print(merged_isochrones_gdf)

/tmp/ipykernel_14203/1338846423.py:39: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  merged_isochrones = pd.concat([merged_isochrones, new_row], ignore_index=True)


Processed isochrone with time 60.0 seconds.
Processed isochrone with time 60.0 seconds.
Processed isochrone with time 60.0 seconds.
Processed isochrone with time 60.0 seconds.
Processed isochrone with time 60.0 seconds.
Processed isochrone with time 60.0 seconds.
Processed isochrone with time 60.0 seconds.
Processed isochrone with time 60.0 seconds.
Processed isochrone with time 60.0 seconds.
Processed isochrone with time 60.0 seconds.
Processed isochrone with time 60.0 seconds.
Processed isochrone with time 60.0 seconds.
Processed isochrone with time 60.0 seconds.
Processed isochrone with time 60.0 seconds.
Processed isochrone with time 60.0 seconds.
Processed isochrone with time 60.0 seconds.
Processed isochrone with time 60.0 seconds.
Processed isochrone with time 60.0 seconds.
Processed isochrone with time 60.0 seconds.
Processed isochrone with time 60.0 seconds.
Processed isochrone with time 60.0 seconds.
Processed isochrone with time 60.0 seconds.
Processed isochrone with time 60

In [64]:
# Function to prepare heatmap data

merged_isochrones_gdf.to_crs(epsg=4326, inplace=True)
def prepare_heatmap_data(merged_isochrones_gdf):
    """
    Prepare heatmap data from merged isochrones.

    Parameters:
    - merged_isochrones_gdf: GeoDataFrame with merged isochrones.

    Returns:
    - List of [latitude, longitude, intensity] for HeatMap.
    """
    heat_data = []
    for _, row in merged_isochrones_gdf.iterrows():
        if 'time' not in row:
            print("Missing 'time' column in merged_isochrones_gdf")
            continue
        centroid = row.geometry.centroid
        lat, lon = centroid.y, centroid.x
        intensity = row['time'] / 60  # Convert time to minutes for intensity
        heat_data.append([lat, lon, intensity])
    return heat_data


In [65]:
heat_data = prepare_heatmap_data(merged_isochrones_gdf)

# Initialize Folium map centered around Zurich
m = folium.Map(location=[47.3769, 8.5417], zoom_start=13)

# Add merged isochrones to the map
for _, row in merged_isochrones_gdf.iterrows():
    folium.GeoJson(
        row['geometry'],
        name=f"Isochrone {row['time']} min",
        style_function=lambda feature: {
            'fillColor': 'blue',
            'color': 'blue',
            'weight': 1,
            'fillOpacity': 0.9,
        }
    ).add_to(m)

# Add recycling collection points to the map using MarkerCluster
marker_cluster = MarkerCluster().add_to(m)
for _, row in rcps.iterrows():
    folium.Marker(
        location=[row.geometry.y, row.geometry.x],
        popup=row['adresse'],
        icon=folium.Icon(color='green', icon='recycle', prefix='fa')
    ).add_to(marker_cluster)

# Create a color map for the heatmap
colormap = cm.LinearColormap(
    colors=['blue', 'green', 'yellow', 'orange', 'red'],
    vmin=0,
    vmax=10
)
colormap.caption = 'Isochrone Time (minutes)'
colormap.add_to(m)

# Save and display the map
heatmap_path = '/home/silas/projects/msc_thesis/data/derived_data/heatmap_test.html'
m.save(heatmap_path)
m